<a href="https://colab.research.google.com/github/kimmunsu/24_langchain_study_colab/blob/main/ms_rag_naver_news_qa_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 설치
!pip install -q langchain langchain-google-genai langchain-community langchain-core langchainhub
!pip install -q faiss-cpu

In [ ]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
import bs4 #web page parsing
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter #text splitting
from langchain_community.document_loaders import WebBaseLoader #web page loading
from langchain_community.vectorstores import FAISS #vector database
from langchain_core.output_parsers import StrOutputParser #output parsing
from langchain_core.runnables import RunnablePassthrough #runnable
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 직원들의 세금 부담을 고려해 정부가 면세해달라는 제안도 나왔습니다.이같은 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

In [ ]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
llms = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llms
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke(
    "부영그룹의 출산 장려 정책에 대해 설명해주세요."
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

'부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하는 출산 장려 정책을 시행하고 있습니다. 연년생이나 쌍둥이 자녀가 있으면 총 2억원을 받을 수 있으며, 셋째까지 낳는 경우 국민주택을 제공합니다. 회사는 직원들의 세금 부담을 고려하여 정부에 출산장려금 면세를 제안했습니다.'

In [ ]:
rag_chain.invoke(
    "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.


'부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원씩, 총 70억원을 지원합니다. 연년생이나 쌍둥이 자녀가 있으면 총 2억원을 받을 수 있습니다. 셋째까지 낳는 경우에는 국민주택을 제공합니다.'

In [ ]:
print(
    rag_chain.invoke("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요.")
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

제공된 텍스트에는 정부의 저출생 대책에 대한 정보가 없습니다. 따라서 저는 정부의 저출생 대책을 bullet points 형식으로 작성할 수 없습니다.


In [ ]:
rag_chain.invoke(
    "부영그룹의 임직원 숫자는 몇명인가요?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.


'제공된 텍스트에는 부영그룹의 임직원 수에 대한 정보가 없습니다.'